# GloVE

Let's work on implementation of GloVE.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# # change the runtime type to GPU Cuda
# import torch

# if torch.cuda.is_available():
#     device = torch.device("cuda")
#     print("GPU is available")
# else:
#     device = torch.device("cpu")
#     print("GPU is not available")



In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/_NLP/NLP-A1-That-s-What-I-LIKE-st125553')

## 1. Load data

In [ ]:
# Load nltk
import nltk

# download news category dataset from nltk
nltk.download('brown') # download brown corpus
nltk.download('punkt') # download punkt for tokenization
nltk.download('punkt_tab') # download punkt_tab for tokenization

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
#1. tokenization
# import the news category dataset
from nltk.corpus import brown
corpus = brown.sents(categories='news')

In [ ]:
#get word sequences and unique words
flatten = lambda l: [item for sublist in l for item in sublist]
vocab = list(set(flatten(corpus)))

In [ ]:
#2. numeralization
#find unique words
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(corpus))) #all the words we have in the system - <UNK>

print(f"before vocabs_len: {len(vocabs)}")
vocabs.append('<UNK>')
print(f"after vocabs_len: {len(vocabs)}")

before vocabs_len: 14394
after vocabs_len: 14395


In [ ]:
#create handy mapping between integer and word
word2index = {v:idx for idx, v in enumerate(vocabs)}
word2index['<UNK>']

14394

In [ ]:

index2word = {v:k for k, v in word2index.items()}
index2word[14394]

'<UNK>'

## 2. Build Co-occurence Matrix X

Here, we need to count the co-occurence of two words given some window size.  We gonna use window size of 1.

In [ ]:
from collections import Counter

X_i = Counter(flatten(corpus))
#X_i

In [ ]:
# window_size = 2
# skip_grams = []

# for doc in corpus:
#     for i in range(window_size, len(doc)-window_size):
#         center = doc[i]
#         outside = [word2index[doc[i+j]] for j in range(-window_size, window_size+1) if i+j != i]
#         for each_out in outside:
#             skip_grams.append((center, each_out))
# #skip_grams

In [ ]:
window_size = 2

skip_grams = []

#loop each corpus
for doc in corpus:
    #look from the 2nd word until second last word
    for i in range(window_size, len(doc)-window_size):
        #center word
        center = doc[i]
        #outside words = 2 words

        outside = []
        for j in range(window_size):
            outside.append(doc[i+(j+1)])
            outside.append(doc[i-(j+1)])

        #for each of these two outside words, we gonna append to a list
        for each_out in outside:
            skip_grams.append((center, each_out))
            #center, outside1;   center, outside2

#skip_grams

In [ ]:
#skip_grams

In [ ]:
X_ik_skipgrams = Counter(skip_grams)
#X_ik_skipgrams

### Weighting function

GloVe includes a weighting function to scale down too frequent words.


In [ ]:
def weighting(w_i, w_j, X_ik):

    #check whether the co-occurences between w_i and w_j is available
    try:
        x_ij = X_ik[(w_i, w_j)]
        #if not exist, then set to 1 "laplace smoothing"
    except:
        x_ij = 1

    #set xmax
    x_max = 100
    #set alpha
    alpha = 0.75

    #if co-ocurrence does not exceeed xmax, then just multiply with some alpha
    if x_ij < x_max:
        result = (x_ij / x_max)**alpha
    #otherwise, set to 1
    else:
        result = 1

    return result

In [ ]:
from itertools import combinations_with_replacement

X_ik = {} #keeping the co-occurences
weighting_dic = {} #already scale the co-occurences using the weighting function

for bigram in combinations_with_replacement(vocab, 2):
    if X_ik_skipgrams.get(bigram):  #if the pair exists in our corpus
        co = X_ik_skipgrams[bigram]
        X_ik[bigram] = co + 1 #for stability
        X_ik[(bigram[1], bigram[0])] = co + 1 #basically apple, banana = banana, apple
    else:
        pass

    weighting_dic[bigram] = weighting(bigram[0], bigram[1], X_ik)
    weighting_dic[(bigram[1], bigram[0])] = weighting(bigram[1], bigram[0], X_ik)

## 3. Prepare train data

In [ ]:
import math

def random_batch(batch_size, word_sequence, skip_grams, X_ik, weighting_dic):

    random_inputs, random_labels, random_coocs, random_weightings = [], [], [], []

    #convert our skipgrams to id
    skip_grams_id = [(word2index[skip_gram[0]], word2index[skip_gram[1]]) for skip_gram in skip_grams]

    #randomly choose indexes based on batch size
    random_index = np.random.choice(range(len(skip_grams_id)), batch_size, replace=False)

    #get the random input and labels
    for index in random_index:
        random_inputs.append([skip_grams_id[index][0]])
        random_labels.append([skip_grams_id[index][1]])
        #coocs
        pair = skip_grams[index] #e.g., ('banana', 'fruit')
        try:
            cooc = X_ik[pair]
        except:
            cooc = 1
        random_coocs.append([math.log(cooc)])

        #weightings
        weighting = weighting_dic[pair]
        random_weightings.append([weighting])

    return np.array(random_inputs), np.array(random_labels), np.array(random_coocs), np.array(random_weightings)

### Testing the method

In [ ]:
#weighting_dic

In [ ]:
skip_grams[0]

('County', 'Grand')

In [ ]:
#skip_grams_id = [(word2index[skip_gram[0]], word2index[skip_gram[1]]) for skip_gram in skip_grams]

In [ ]:
batch_size = 2
x, y, cooc, weighting = random_batch(batch_size, corpus, skip_grams, X_ik, weighting_dic)

In [ ]:
x

array([[10204],
       [12718]])

In [ ]:
y

array([[14132],
       [11992]])

In [ ]:
cooc

array([[1.09861229],
       [1.09861229]])

In [ ]:
weighting

array([[0.07208434],
       [0.07208434]])

## 4. Model


In [ ]:
class Glove(nn.Module):

    def __init__(self, voc_size, emb_size, word2index):
        super(Glove, self).__init__()
        self.center_embedding  = nn.Embedding(voc_size, emb_size)
        self.outside_embedding = nn.Embedding(voc_size, emb_size)

        self.center_bias       = nn.Embedding(voc_size, 1)
        self.outside_bias      = nn.Embedding(voc_size, 1)

        self.word2index = word2index

    def forward(self, center, outside, coocs, weighting):
        center_embeds  = self.center_embedding(center) #(batch_size, 1, emb_size)
        outside_embeds = self.outside_embedding(outside) #(batch_size, 1, emb_size)

        center_bias    = self.center_bias(center).squeeze(1)
        target_bias    = self.outside_bias(outside).squeeze(1)

        inner_product  = outside_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #(batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) = (batch_size, 1)

        loss = weighting * torch.pow(inner_product + center_bias + target_bias - coocs, 2)

        return torch.sum(loss)

    def get_embed(self, word):
        try:
            index = word2index[word]
        except:
            index = word2index['<UNK>']

        word = torch.LongTensor([index])

        embed_c = self.center_embedding(word)
        embed_o = self.outside_embedding(word)
        embed   = (embed_c + embed_o) / 2

        return embed[0][0].item(), embed[0][1].item()

In [ ]:
len(vocabs)

14395

In [ ]:
#test our system

voc_size = len(vocabs)
emb_size = 2
model = Glove(voc_size, emb_size,word2index)

In [ ]:
x_tensor = torch.LongTensor(x)
y_tensor = torch.LongTensor(y)
cooc_tensor = torch.FloatTensor(cooc)
weighting_tensor = torch.FloatTensor(weighting)

In [ ]:
loss = model(x_tensor, y_tensor, cooc_tensor, weighting_tensor)

In [ ]:
loss

tensor(1.1692, grad_fn=<SumBackward0>)

## 5. Training

In [ ]:
batch_size     = 2 # mini-batch size
embedding_size = 2 #so we can later plot
model          = Glove(voc_size, embedding_size, word2index)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import time
start_time = time.time()

num_epochs = 1000
for epoch in range(num_epochs):

    start = time.time()

    input_batch, target_batch, cooc_batch, weighting_batch = random_batch(batch_size, corpus, skip_grams, X_ik, weighting_dic)
    input_batch  = torch.LongTensor(input_batch)         #[batch_size, 1]
    target_batch = torch.LongTensor(target_batch)        #[batch_size, 1]
    cooc_batch   = torch.FloatTensor(cooc_batch)         #[batch_size, 1]
    weighting_batch = torch.FloatTensor(weighting_batch) #[batch_size, 1]

    optimizer.zero_grad()
    loss = model(input_batch, target_batch, cooc_batch, weighting_batch)

    loss.backward()
    optimizer.step()

    end = time.time()

    epoch_mins, epoch_secs = epoch_time(start, end)

    if (epoch + 1) % 100 == 0:

      end = time.time()
      epoch_mins, epoch_secs = epoch_time(start, end)

      print(f"Epoch: {epoch + 1} | Loss: {loss:.6f} | time: {epoch_mins}m {epoch_secs}s")
      start = end

end_time = time.time()
minutes, seconds = epoch_time(start_time, end_time)
print(f"Total time: {minutes:2.0f} minutes {seconds:2.0f} seconds")


Epoch: 100 | Loss: 0.208598 | time: 0m 0s
Epoch: 200 | Loss: 0.256442 | time: 0m 0s
Epoch: 300 | Loss: 0.999286 | time: 0m 0s
Epoch: 400 | Loss: 21.913521 | time: 0m 0s
Epoch: 500 | Loss: 0.057508 | time: 0m 0s
Epoch: 600 | Loss: 0.088874 | time: 0m 0s
Epoch: 700 | Loss: 0.659058 | time: 0m 0s
Epoch: 800 | Loss: 1.040385 | time: 0m 0s
Epoch: 900 | Loss: 0.245381 | time: 0m 0s
Epoch: 1000 | Loss: 39.603188 | time: 0m 0s
Total time:  1 minutes 21 seconds


## 6. Testing

In [ ]:
vect = []

for word in vocabs:
    vect.append(model.get_embed(word))
vect = np.array(vect)

In [ ]:
# # ipython-input-54-07b52817ebdf
# vect = []

# for word in vocabs:
#     try:
#         vect.append(model.get_embed(word)) # Get embedding for the current word
#     except IndexError:
#         # Handle IndexError, likely due to the word index being out of range
#         # This could happen if the word is not in the model's vocabulary
#         print(f"Word '{word}' caused an IndexError. Skipping...")
#         # Alternatively, you could assign a default embedding for unknown words
#         # For example: vect.append(model.get_embed('<UNK>'))
# vect = np.array(vect) # Convert the list of embeddings to a numpy array

In [ ]:
#scipy version
from scipy import spatial

def cos_sim(a, b):
    cos_sim = 1 - spatial.distance.cosine(a, b)  #distance = 1 - similarlity, because scipy only gives distance
    return cos_sim

def cos_sim_scores(vect_space, target_vect):
    scores = []
    for each_vect in vect_space:
        each_vect = tuple(each_vect)
        target_vect=tuple(target_vect)
        scores.append(cos_sim(target_vect, each_vect))

    return np.array(scores)

In [ ]:
def similarity(model, data):
    words = data.split(" ")

    embed0 = np.array(model.get_embed(words[0]))
    embed1 = np.array(model.get_embed(words[1]))
    embed2 = np.array(model.get_embed(words[2]))

    sim_vect = embed1 - embed0 + embed2

    sim_scores = cos_sim_scores(vect, sim_vect)
    max_score_idx = np.argmax(sim_scores)
    sim_word = index2word[max_score_idx]

    result = False
    if sim_word == words[3]:
        result = True

    return result

### Semantic Test

In [ ]:
semantic_file = "data/word-test-semantic.txt"
# open file
with open(semantic_file, "r") as file:
    sem_file = file.readlines()
    #send semantic into vector

semantic = []
for sent in sem_file:
    semantic.append(sent.strip())

#semantic

In [ ]:
sem_count = len(semantic)
#sem_total
sem_correct = 0
for sent in semantic:
    if similarity(model, sent):
        sem_correct += 1

In [ ]:
sem_accuracy = sem_correct / sem_count
print(f"Semantic accuracy: {sem_accuracy:2.2f}")
print(f"Semantic correct: {sem_correct}")
print(f"Semantic count: {sem_count}")

Semantic accuracy: 0.00
Semantic correct: 0
Semantic count: 506


### Syntatic Test

In [ ]:
syntatic_file = "data/word-test-syntatic.txt"
# open file
with open(syntatic_file, "r") as file:
    syn_file = file.readlines()

syntatic = []
for sent in syn_file:
    syntatic.append(sent.strip())
#syntatic

In [ ]:
syn_count = len(syntatic)
syn_correct = 0
for sent in syntatic:
    if similarity(model, sent):
        syn_correct += 1

In [ ]:
syn_accuracy = syn_correct / syn_count
print(f"Syntatic accuracy: {syn_accuracy:2.2f}")
print(f"Syntatic correct: {syn_correct}")
print(f"Syntatic count: {syn_count}")

Syntatic accuracy: 0.00
Syntatic correct: 0
Syntatic count: 1560


### Similarity Test


In [ ]:
similarity_file = "data/wordsim353_sim_rel/wordsim_similarity_goldstandard.txt"
# open file
with open(similarity_file, "r") as file:
    sim_file = file.readlines()

similarity = []
for sent in sim_file:
    similarity.append(sent.strip())
#syntatic

In [ ]:
def similarity_test(model, test_data):
    words = test_data.split("\t")

    embed0 = np.array(model.get_embed(words[0].strip()))
    embed1 = np.array(model.get_embed(words[1].strip()))

    model_result = embed1 @ embed0.T
    sim_result = float(words[2].strip())

    return sim_result, model_result

In [ ]:
sim_scores = []
model_scores = []
for sent in similarity:
    sim_result, model_result = similarity_test(model, sent)

    sim_scores.append(sim_result)
    model_scores.append(model_result)

In [ ]:
from scipy.stats import spearmanr

corr = spearmanr(sim_scores, model_scores)[0]

print(f"The correlation result is {corr:2.2f}.")

The correlation result is 0.06.


### Test P Value

In [ ]:
import pandas as pd
from scipy.stats import spearmanr

# Load the dataset
file_path = "data/wordsim353_sim_rel/wordsim_similarity_goldstandard.txt"
similarity_data = pd.read_csv(file_path, sep='\t', names=['word1', 'word2', 'similarity'])

# Display sample data
print(similarity_data.head())


        word1  word2  similarity
0       tiger    cat        7.35
1       tiger  tiger       10.00
2       plane    car        5.77
3       train    car        6.31
4  television  radio        6.77


In [ ]:
def compute_dot_product(model, word1, word2):
    """Compute dot product between embeddings of two words."""
    embedding1 = model.get_embed(word1)
    embedding2 = model.get_embed(word2)
    if isinstance(embedding1, tuple) or isinstance(embedding2, tuple):
        embedding1 = list(embedding1)
        embedding2 = list(embedding2)
    return np.dot(embedding1, embedding2)

# Compute model-based similarities
def compute_model_similarities(model, data):
    model_similarities = []
    for _, row in data.iterrows():
        try:
            dot_product = compute_dot_product(model, row['word1'], row['word2'])
            model_similarities.append(dot_product)
        except KeyError:
            # Handle missing words in the model
            model_similarities.append(0.0)
    return model_similarities


In [ ]:
# Prepare data for word embedding models
model_similarities = compute_model_similarities(model, similarity_data)
gold_standard_similarities = similarity_data['similarity'].values

# Calculate Spearman's rank correlation
correlation, p_value = spearmanr(model_similarities, gold_standard_similarities)

print(f"Spearman correlation: {correlation:.4f}, p-value: {p_value:.4f}")


Spearman correlation: 0.0583, p-value: 0.4084



## 7. Saving the model

In [ ]:
# Saving the model for testing
torch.save(model.state_dict(), 'app/models/glove.model')

In [ ]:
glove_args = {
    'voc_size': voc_size,
    'emb_size': emb_size,
    'word2index': word2index,
}


In [ ]:
import pickle
pickle.dump(glove_args, open('app/models/glove.args', 'wb'))

In [ ]:
glove_args = pickle.load(open('app/models/glove.args', 'rb'))
model_glove = Glove(**glove_args)
model_glove.load_state_dict(torch.load('app/models/glove.model'))

# Test the model
model_glove.get_embed('The')

<ipython-input-56-8fdb8e7de74c>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_glove.load_state_dict(torch.load('app/models/glove.model'))


(-0.6850587129592896, 0.10881073772907257)

## 8. Calculate MSE

In [58]:
from sklearn.metrics import mean_squared_error
import pandas as pd
from scipy.stats import spearmanr
import numpy as np

# Load the dataset
file_path = "data/wordsim353_sim_rel/wordsim_similarity_goldstandard.txt"
similarity_data = pd.read_csv(file_path, sep='\t', names=['word1', 'word2', 'similarity'])

def compute_dot_product(model, word1, word2):
    """Compute dot product between embeddings of two words."""
    embedding1 = model.get_embed(word1)
    embedding2 = model.get_embed(word2)
    if embedding1 is None or embedding2 is None:
        return 0.0  # Default to zero for missing embeddings
    return np.dot(np.array(embedding1), np.array(embedding2))

def compute_model_similarities(model, data):
    """Compute similarities using the model."""
    model_similarities = []
    for _, row in data.iterrows():
        dot_product = compute_dot_product(model, row['word1'], row['word2'])
        model_similarities.append(dot_product)
    return model_similarities

# Prepare data for word embedding models
gold_standard_similarities = similarity_data['similarity'].values  # Y-true
model_similarities = compute_model_similarities(model, similarity_data)

# Calculate Spearman's rank correlation
correlation, p_value = spearmanr(model_similarities, gold_standard_similarities)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(gold_standard_similarities, model_similarities)
ytrue_mse = mean_squared_error(gold_standard_similarities, gold_standard_similarities)

# Output results
print(f"Spearman correlation: {correlation:.4f}, p-value: {p_value:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}, Yture MSE: {ytrue_mse:.4f}")

# Calculate correlation for Y-True
ytrue_correlation, ytrue_p_value = spearmanr(gold_standard_similarities, gold_standard_similarities)
print(f"Correlation for Y-True: {ytrue_correlation:.4f}, p-value: {ytrue_p_value:.4f}")


Spearman correlation: 0.0583, p-value: 0.4084
Mean Squared Error (MSE): 30.8755, Yture MSE: 0.0000
Correlation for Y-True: 1.0000, p-value: 0.0000
